In [7]:
import asyncio
import json
import os
from datetime import datetime
import pytz
from websockets import connect
from termcolor import cprint

In [13]:
#list of symbols to track
symbols=['btcusdt','ethusdt','solusdt','bnbusdt','dogeusdt','wifusdt']
websocket_url_base='wss://fstream.binance.com/ws/'
trades_filename='binance_trades.csv'

In [27]:
import os
import json
import asyncio
from datetime import datetime

import pytz
from termcolor import cprint
from websockets import connect  # pip install websockets

symbols=['btcusdt','ethusdt','solusdt','bnbusdt','dogeusdt','wifusdt']
websocket_url_base='wss://fstream.binance.com/ws/'
trades_filename='binance_trades.csv'

trades_filename = "binance_trades.csv"

# Check if the CSV file exists, if not create it with header
if not os.path.isfile(trades_filename):
    with open(trades_filename, "w") as f:
        f.write(
            "Event Time,Symbol,Aggregate Trade ID,Price,Quantity,Trade Time,Is Buyer Maker\n"
        )


async def binance_trade_stream(uri: str, symbol: str, filename: str) -> None:
    """
    Connects to a Binance aggTrade stream for a single symbol and logs large trades.
    """
    async with connect(uri) as websocket:
        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)  # fixed typo: mnessage -> message

                # Basic parsing (aggTrade format)
                event_time = int(data["E"])
                agg_trade_id = data["a"]
                price = float(data["p"])
                quantity = float(data["q"])
                trade_time = int(data["T"])
                is_buyer_maker = data["m"]

                est = pytz.timezone("US/Eastern")
                readable_trade_time = datetime.fromtimestamp(
                    trade_time / 1000, est
                ).strftime("%H:%M:%S")

                usd_size = price * quantity  # fixed us_size / usd_size mismatch
                display_symbol = symbol.upper().replace("USDT", "")

                # Only care about trades above a certain USD size
                if usd_size > 14999:
                    trade_type = "SELL" if is_buyer_maker else "BUY"
                    color = "red" if trade_type == "SELL" else "green"

                    stars = ""
                    attrs = []

                    # Tiered styling by size (you can adjust thresholds as you like)
                    if usd_size >= 100000:
                        stars = "***"
                        attrs = ["bold"]
                        repeat_count = 3
                    elif usd_size >= 50000:
                        stars = "**"
                        attrs = ["bold"]
                        repeat_count = 2
                    else:
                        repeat_count = 1

                    # Optional: change colors for very large trades
                    if usd_size >= 50000:
                        if trade_type == "SELL":
                            color = "magenta"
                        else:
                            color = "blue"

                    output = (
                        f"{stars} {trade_type} {display_symbol} "
                        f"{readable_trade_time} ${usd_size:,.0f}"
                    )

                    for _ in range(repeat_count):
                        cprint(output, "white", f"on_{color}", attrs=attrs)

                    # Log to CSV
                    with open(filename, "a") as f:
                        f.write(
                            f"{event_time},{symbol.upper()},{agg_trade_id},"
                            f"{price},{quantity},{trade_time},{is_buyer_maker}\n"
                        )

            except Exception as e:  # fixed: Exception, not Exceptions
                cprint(f"Error in stream for {symbol}: {e}", "white", "on_red")
                await asyncio.sleep(5)


async def main():
    filename = trades_filename

    tasks = []
    for symbol in symbols:
        stream_url = f"{websocket_url_base}{symbol}@aggTrade"
        # fixed: sumbol -> symbol, function expects `symbol`, not `symbols`
        tasks.append(binance_trade_stream(stream_url, symbol, filename))

    await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(main())

 BUY WIF 09:35:27 $18,770
 SELL SOL 09:35:28 $30,146
 BUY ETH 09:35:29 $48,528
 BUY SOL 09:35:30 $19,939
 BUY SOL 09:35:30 $30,008
 SELL SOL 09:35:31 $30,769
** SELL SOL 09:35:32 $61,542
** SELL SOL 09:35:32 $61,542


CancelledError: 